Peddle Peddle for object detection

In [ ]:
!pip install paddlepaddle-gpu==2.4.2.post117 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html


Looking in links: https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html


We  are using peddle peddle on cuda we need to install paddlepaddle-gpu. so, its command for gogle colab

In [ ]:
!pip install --upgrade setuptools

In [ ]:
import paddle
paddle.utils.run_check()

 Original Error is 


Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 CPU.
PaddlePaddle works well on 2 CPUs.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.


for using Paddle paddle clone the repository from github as

In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleDetection.git

Cloning into 'PaddleDetection'...
remote: Enumerating objects: 257131, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 257131 (delta 9), reused 6 (delta 1), pack-reused 257112
Receiving objects: 100% (257131/257131), 421.29 MiB | 33.53 MiB/s, done.
Resolving deltas: 100% (209712/209712), done.


In [ ]:
cd PaddleDetection

/content/PaddleDetection/PaddleDetection/PaddleDetection/PaddleDetection/PaddleDetection/PaddleDetection


Import the necessary libraries

In [ ]:
import os
from urllib.request import urlretrieve
from zipfile import ZipFile
import shutil
import numpy as np
import yaml
import cv2
from matplotlib import pyplot as plt
from PIL import Image,ImageDraw, ImageFile, ImageFont
import paddle

import math
import matplotlib
import matplotlib.pyplot as plt

Define some helper function that will be used later for downloading the necessary files

In [ ]:
import os
from urllib.request import urlretrieve
from zipfile import ZipFile

def download_and_unzip(url, save_path):
    print("Downloading and extracting assets... ", end="")
    urlretrieve(url, save_path)
    try:
        with ZipFile(save_path) as z:
            z.extractall(os.path.split(save_path)[0])
        print("Done")
    except Exception as e:
        print("\nInvalid file.", e)




Use predefined helper function

In [ ]:
import os

URL = r"https://www.dropbox.com/s/a6nzj521t5woxp7/dtection_pose.zip?dl=1"
asset_zip_path = os.path.join(os.getcwd(), "segmentation.zip")

if not os.path.exists(asset_zip_path):
    download_and_unzip(URL, asset_zip_path)


Install the requirements as

In [ ]:
pip install -r requirements.txt

It will create a file and that will contain
1. model weights (model.pdiparams)
2. contain information about parameter (model.pdiparams.info)
3. conttain model structure description (model.pdmodel)
4. preprcessing the configuration file  (infer_cfg.yml)

In [ ]:
!python tools/export_model.py -c configs/dino/dino_r50_4scale_1x_coco.yml --output_dir=./inference_model \
 -o weights=https://paddledet.bj.bcebos.com/models/dino_r50_4scale_1x_coco.pdparams

/usr/local/lib/python3.10/dist-packages/setuptools/sandbox.py:13: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2868: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2868: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/u

Import necessary function

In [ ]:
from deploy.python.keypoint_infer import KeyPointDetector
from deploy.python.infer import Detector, PredictConfig
from deploy.python.visualize import visualize_pose, get_color_map_list, visualize_box_mask, get_color

Since the font size is hardcoded in the file making the text nearly impossible to read for high resoltion images, thus we would write our own code for plotting the results. for this using follwing function


In [ ]:
def draw_box(im, np_boxes, labels, threshold=0.5):
    im = Image.fromarray(im)
    np_boxes=np_boxes['boxes']
    draw_thickness = max(round(sum(im.size) / 2 * 0.01), 2)
    draw = ImageDraw.Draw(im)
    clsid2color = {}
    color_list = get_color_map_list(len(labels))
    expect_boxes = (np_boxes[:, 1] > threshold) & (np_boxes[:, 0] > -1)
    np_boxes = np_boxes[expect_boxes, :]

    for dt in np_boxes:
        clsid, bbox, score = int(dt[0]), dt[2:], dt[1]
        if clsid not in clsid2color:
            clsid2color[clsid] = color_list[clsid]
        color = tuple(clsid2color[clsid])

        xmin, ymin, xmax, ymax = bbox
        draw.line(
            [(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin), (xmin, ymin)],
            width=draw_thickness // 2,
            fill=color,
        )
        font = ImageFont.truetype(
            "LiberationSans-Regular.ttf", draw_thickness * 2
        )

        # draw label
        text = "{} {:.4f}".format(labels[clsid], score)
        new_box = draw.textbbox((0, 0), text, font=font)
        tw = new_box[2] - new_box[0]  # bottom-top
        th = new_box[3] - new_box[1]  # right-left
        draw.rectangle([(xmin + 1, ymin - th), (xmin + tw + 1, ymin)], fill=color)
        draw.text((xmin + 1, ymin - th), text, fill=(255, 255, 255), font=font)
    return im

In [ ]:
!python deploy/python/infer.py --model_dir=./inference_model/dino_r50_4scale_1x_coco --image_file="object-detection_input_image.jpg" --device=GPU


/usr/local/lib/python3.10/dist-packages/setuptools/sandbox.py:13: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2868: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2868: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/u

In [ ]:

!python deploy/python/infer.py --model_dir=./inference_model/dino_r50_4scale_1x_coco --video_file=object-detection.mp4 --device=GPU


/usr/local/lib/python3.10/dist-packages/setuptools/sandbox.py:13: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2868: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2868: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.cloud')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/u

In [ ]:
def video_inference(video_file, detector, keypoint_or_object, output_name):
  capture = cv2.VideoCapture(video_file)
  width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = int(capture.get(cv2.CAP_PROP_FPS))
  out_path = output_name
  fourcc = cv2.VideoWriter_fourcc(*"mp4v")
  writer = cv2.VideoWriter(out_path, fourcc, fps, (width, height))
  index = 1
  while 1:
      ret, frame = capture.read()
      if not ret:
          break
      print("detect frame: %d" % (index))
      index += 1
      results = detector.predict_image([frame[..., ::-1]], visual=False)
      if keypoint_or_object=="object":
        im = visualize_box_mask(
            frame, results, PredictConfig(model_dir).labels, threshold=0.5
        )
      else:
        im_results = {}
        im_results["keypoint"] = [results["keypoint"], results["score"]]
        im = visualize_pose(frame, im_results, visual_thresh=0.3, returnimg=True)

      im = np.array(im)
      writer.write(im)

  writer.release()

In [ ]:
video_inference("http://192.168.1.104:8080/video", detector, "object", "objects_in_video.mp4")
